In [ ]:
!pip install langchain faiss-cpu sentence-transformers pypdf openai tiktoken

In [89]:
from google.colab import files
uploaded = files.upload()


Saving college_faq.pdf to college_faq (2).pdf
Saving academic_calendar.pdf to academic_calendar (2).pdf
Saving syllabus (1).pdf to syllabus (1) (2).pdf


In [90]:
from langchain_community.document_loaders import PyPDFLoader

pdf_files = list(uploaded.keys())
documents = []

for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    documents.extend(loader.load())

print(f"Loaded {len(documents)} pages")

Loaded 3 pages


In [91]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs = text_splitter.split_documents(documents)
print(f"Total chunks created: {len(docs)}")

Total chunks created: 6


In [92]:
# NEW IMPORT PATH
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

In [93]:
from langchain_community.vectorstores import FAISS

vector_db = FAISS.from_documents(docs, embeddings)
vector_db.save_local("college_faiss_index")

print("Vector database created successfully")

Vector database created successfully


In [94]:
vector_db = FAISS.load_local(
    "college_faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

In [95]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

model_id = "google/flan-t5-base"

print(f"Downloading {model_id} to run locally")

#Load Tokenizer and Model locally
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

#Create a local pipeline
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.2
)

llm = HuggingFacePipeline(pipeline=pipe)

print("LLM Initialized locally!")

Device set to use cpu


LLM Initialized locally!


In [96]:
def college_chatbot(query):
    # Retrieve relevant docs
    relevant_docs = vector_db.similarity_search(query, k=3)

    context = ""
    for doc in relevant_docs:
        context += doc.page_content + "\n\n"

    prompt = f"""
    You are a helpful college assistant.
    Answer the question strictly using the context below.
    If the answer is not available, say "Information not available".

    Context:
    {context}

    Question:
    {query}
    """

    return llm.invoke(prompt)


In [97]:
question = "What scholarships are available?"
answer = college_chatbot(question)

print("Answer:\n", answer)


Answer:
 merit-based, need-based, and government scholarships


In [98]:
question = "When are the results declared?"
answer = college_chatbot(question)

print("Answer:\n", answer)

Answer:
 June
